# Exploring and Visualizing Beyond the East Busway Data
## Matthew Samach

## Reading in Data Using Pandas

In [3]:
raw_data = pd.read_csv("../Data/responses_anonymous.csv", encoding = "ISO-8859-1")
raw_data.head(1)

,id,addr,timestamp,3 - Area of interest,4 - Communities of interest,5 - Destinations,6 - Corridor preference,7 - Pittsburgh areas,8 - Allegheny County areas,9 - PAAC Routes Used,10 - Routing Type Preference,11 - Current Transit Usage,12 - Other Transport Used,12 - Why Other Transport Used,13 - Age,13 - Employment status,13 - Where From,14 - Gender,14 - Race,15 - Comments
0,426926b2-be9a-4d8e-8aa6-423714dee82f,172.58.43.140,2019-08-03T18:35:41,"['McKeesport and the Mon Valley', 'Monroeville...","['McKeesport', 'Monroeville']","['Aldi Store #97', 'Centers for Rehab Services...",['Mon Valley via Westinghouse Bridge'],['Oakland (Pittsburgh)'],[],['P1 - East Busway - All Stops'],['2'],"['Rarely, maybe a couple times a year']",['Private car'],"[""Transit isn't reliable to get me where I'm g...",['65-74 years old'],['Full-time employment'],['Regent Square (Pittsburgh)'],['Man'],['Black or African American'],['']


In [109]:
county_gis = pd.read_csv('../Data/Allegheny_County_Municipal_Boundaries.csv')
county_gis.head(1)

,OBJECTID,NAME,TYPE,LABEL,COG,SCHOOLD,CONGDIST,FIPS,REGION,ACRES,SQMI,MUNICODE,CNTL_ID,CNTYCOUNCIL,EOC,ASSESSORTERRITORY,VALUATIONAREA,YEARCONVERTED,GlobalID
0,1,CHESWICK,BOROUGH,Cheswick Borough,Allegheny Valley North,Allegheny Valley,4,13392,NH,350.191284,0.547174,815,3100,7,NEWCOM,East,Alle-Kiski Valley,1966,f29648dc-0d4f-4e35-8f2d-7b465dcff308


In [53]:
pgh_gis = pd.read_csv('../Data/pgh_neighborhoods.csv')
pgh_gis.head(2)

,objectid,fid_blockg,statefp10,countyfp10,tractce10,blkgrpce10,geoid10,namelsad10,mtfcc10,funcstat10,...,shape_le_1,shape_ar_1,page_number,plannerassign,created_user,created_date,last_edited_user,last_edited_date,Shape__Area,Shape__Length
0,1,0.0,42.0,3.0,40500.0,1.0,4.200304e+11,Block Group 1,G5030,S,...,11526.863222,7.843108e+06,15,Derek Dauphin,pgh.admin,2019-10-23T14:17:16.403Z,pgh.admin,2019-10-23T14:17:16.403Z,7.842997e+06,11525.904546
1,2,1.0,42.0,3.0,40400.0,1.0,4.200304e+11,Block Group 1,G5030,S,...,20941.386025,1.390463e+07,15,Derek Dauphin,pgh.admin,2019-10-23T14:17:16.403Z,pgh.admin,2019-10-23T14:17:16.403Z,1.390469e+07,20945.562570


In [5]:
pivot = pd.read_csv("../Data/pivot_anon_source.csv", encoding = "ISO-8859-1")
pivot.head(2)

,id,timestamp,question,answer
0,009a7c74-36df-485b-b9ab-4d89d73ae94c,2019-08-23T12:16:01,3 - Area of interest,McKeesport and the Mon Valley
1,009a7c74-36df-485b-b9ab-4d89d73ae94c,2019-08-23T12:16:01,4 - Communities of interest,Duquesne


Data are not in First Normal Form (there are multiple values per cell in some cases). Will have to write a function to move data into that form.

In [6]:
data = helpers.clean(raw_data)
data.head(1)

,id,addr,timestamp,3 - Area of interest,4 - Communities of interest,5 - Destinations,6 - Corridor preference,7 - Pittsburgh areas,8 - Allegheny County areas,9 - PAAC Routes Used,10 - Routing Type Preference,11 - Current Transit Usage,12 - Other Transport Used,12 - Why Other Transport Used,13 - Age,13 - Employment status,13 - Where From,14 - Gender,14 - Race,15 - Comments
0,426926b2-be9a-4d8e-8aa6-423714dee82f,172.58.43.140,2019-08-03T18:35:41,McKeesport and the Mon Valley| Monroeville and...,McKeesport| Monroeville,Aldi Store #97| Centers for Rehab Services For...,Mon Valley via Westinghouse Bridge,Oakland (Pittsburgh),,P1 - East Busway - All Stops,2,"Rarely, maybe a couple times a year",Private car,Transit isnt reliable to get me where Im going...,65-74 years old,Full-time employment,Regent Square (Pittsburgh),Man,Black or African American,


In [7]:
data['13 - Where From']

0            Regent Square (Pittsburgh)
1      Squirrel Hill South (Pittsburgh)
2                  Forest Hills Borough
3                        Rankin Borough
4                  West Mifflin Borough
                     ...               
513                Turtle Creek Borough
514                 Wilkinsburg Borough
515                   Swissvale Borough
516     Point Breeze North (Pittsburgh)
517           East Liberty (Pittsburgh)
Name: 13 - Where From, Length: 518, dtype: object

In [8]:
whereHot = helpers.oneHot(dFrame=data, col='13 - Where From')
whereHot.head(1)

13 - Where From col has 103 unique vals


,id,addr,timestamp,3 - Area of interest,4 - Communities of interest,5 - Destinations,6 - Corridor preference,7 - Pittsburgh areas,8 - Allegheny County areas,9 - PAAC Routes Used,...,White Oak Borough,Whitehall Borough,Wilkins Township,Wilkinsburg Borough,Wilmerding Borough,mck,mon valley,portVue,southside,westend
0,426926b2-be9a-4d8e-8aa6-423714dee82f,172.58.43.140,2019-08-03T18:35:41,McKeesport and the Mon Valley| Monroeville and...,McKeesport| Monroeville,Aldi Store #97| Centers for Rehab Services For...,Mon Valley via Westinghouse Bridge,Oakland (Pittsburgh),,P1 - East Busway - All Stops,...,0,0,0,0,0,0,0,0,0,0


In [9]:
mult_col_dict = {3 : '3 - Area of interest', 
                4 : '4 - Communities of interest', 
                5 : '5 - Destinations', 
                7 : '7 - Pittsburgh areas', 
                8 : '8 - Allegheny County areas', 
                9 : '9 - PAAC Routes Used', 
                10 : '10 - Routing Type Preference', 
                12 : '12 - Other Transport Used', 
                13 : '12 - Why Other Transport Used', 
                16 : '13 - Where From'}

In [10]:
raw_data.columns[list(mult_col_dict.keys())]

Index(['3 - Area of interest', '4 - Communities of interest',
       '5 - Destinations', '7 - Pittsburgh areas',
       '8 - Allegheny County areas', '9 - PAAC Routes Used',
       '10 - Routing Type Preference', '12 - Other Transport Used',
       '12 - Why Other Transport Used', '13 - Where From'],
      dtype='object')

In [12]:
def find_unique_loc(dFrame, col):
    #locDict = {}
    locs = []
    for i in range(dFrame.shape[0]):
        cell_list = [x.strip() for x in dFrame[col].loc[i].split("|")]
        locs = locs + cell_list
    return(set(locs))

In [52]:
from_loc = find_unique_loc(data, '13 - Where From')
from_pgh_loc = {x for x in from_loc if re.search('Pittsburgh', x, re.IGNORECASE)}
from_pgh_loc.update(['southside','westend'])
from_out_loc = from_loc - from_pgh_loc
print('{0} PGH neigborhoods, {1} non-PGH municipalities'.format(len(from_pgh_loc), len(from_out_loc)))

46 PGH neigborhoods, 57 non-PGH municipalities


In [137]:
def countyMatcher(placelist, gisTown, gisType, gisAll):
    ''' 
    Takes in a location column and returns a matcher to a GIS asset
    Matches on the strict name, then finds municipality type
    '''
    # List for dicts for easy dataframe creation
    dict_list = []

    unusual_match = {'2434 south braddock ave': ('SWISSVALE', 'Swissvale Borough', 'BOROUGH'),
                     'mck': ('MCKEESPORT','McKeesport', 'CITY'),
                     'mon valley': ('','',''),
                     'Wexford': ('PINE', 'Pine Township', 'TOWNSHIP'),
                     'Bethel Park Borough': ('BETHEL PARK', 'Bethel Park Municipality', 'MUNICIPALI'),
                     'Pittsburgh': ('PITTSBURGH', 'Pittsburgh', 'CITY')}
    # extracting municipality type
    p = re.compile("(Township|Borough|Municipality|City)")
    # Iterating over nonpgh places
    for place in placelist:
        # New dict for storing data
        dict_ = {}
        # Find muni type first

        # Replace pittsburgh:
        if re.search('Pittsburgh', place, re.IGNORECASE):
            mName = (unusual_match['Pittsburgh'][0], 100)
            aMatch = (unusual_match['Pittsburgh'][1], 100)
            tMatch = (unusual_match['Pittsburgh'][2], 100)
        # Take out messy matches o
        elif place in unusual_match.keys():
            mName = (unusual_match[place][0], 100)
            aMatch = (unusual_match[place][1], 100)
            tMatch = (unusual_match[place][2], 100)
        # Use our method to find best match, we can set a threshold here
        else:
            type_result = p.search(place, re.IGNORECASE)
            if type_result:
                muni_type = type_result.group(1)
                tMatch = process.extractOne(muni_type, gisType, scorer=fuzz.ratio)
            else:
                tMatch = ('', 100)
            mName = process.extractOne(place, gisTown,  scorer=fuzz.ratio)#, score_cutoff = 60)
            aMatch = process.extractOne(place, gisAll,  scorer=fuzz.ratio)
        
        dict_.update({"from_ppt" : place})
        dict_.update({"from_LabelGIS" : aMatch[0]})
        dict_.update({"labelScore" : aMatch[1]})
        dict_.update({"from_NameGIS" : mName[0]})
        dict_.update({"nameScore" : mName[1]})
        dict_.update({"from_TypeGIS" : tMatch[0]})
        dict_.update({"typeScore" : tMatch[1]})
        dict_list.append(dict_)

    matches_all = pd.DataFrame(dict_list)
    return(matches_all)

In [138]:
fromCounty = countyMatcher(from_loc, county_gis.NAME, county_gis.TYPE, county_gis.LABEL)
fromCounty

,from_ppt,from_LabelGIS,labelScore,from_NameGIS,nameScore,from_TypeGIS,typeScore
0,Sheraden (Pittsburgh),Pittsburgh,100,PITTSBURGH,100,CITY,100
1,portVue,Port Vue Borough,61,PORT VUE,93,,100
2,Morningside (Pittsburgh),Pittsburgh,100,PITTSBURGH,100,CITY,100
3,Trafford Borough,Trafford Borough,100,TRAFFORD,67,BOROUGH,100
4,Crafton Borough,Crafton Borough,100,CRAFTON,64,BOROUGH,100
...,...,...,...,...,...,...,...
98,Allegheny Center (Pittsburgh),Pittsburgh,100,PITTSBURGH,100,CITY,100
99,Munhall Borough,Munhall Borough,100,MUNHALL,64,BOROUGH,100
100,Marshall-Shadeland (Pittsburgh),Pittsburgh,100,PITTSBURGH,100,CITY,100
101,Edgeworth Borough,Edgeworth Borough,100,EDGEWORTH,69,BOROUGH,100


In [122]:
county_gis[['NAME', 'LABEL', 'TYPE', 'COG', 'FIPS', 'MUNICODE', 'OBJECTID', 'REGION', 'SQMI', 'ACRES']]

,NAME,LABEL,TYPE,COG,FIPS,MUNICODE,OBJECTID,REGION,SQMI,ACRES
0,CHESWICK,Cheswick Borough,BOROUGH,Allegheny Valley North,13392,815,1,NH,0.547174,350.191284
1,EDGEWORTH,Edgeworth Borough,BOROUGH,Quaker Valley,22576,824,4,AA,1.610538,1030.743774
2,SEWICKLEY,Sewickley Borough,BOROUGH,Quaker Valley,69376,851,5,AA,1.122359,718.309570
3,SEWICKLEY HEIGHTS,Sewickley Heights Borough,BOROUGH,,69400,869,6,AA,7.404672,4738.989746
4,HARMAR,Harmar Township,TOWNSHIP,Allegheny Valley North,32624,915,8,NH,6.345092,4060.858643
...,...,...,...,...,...,...,...,...,...,...
125,KILBUCK,Kilbuck Township,TOWNSHIP,Quaker Valley,39624,920,31505,NH,2.650067,1696.042969
126,EAST MCKEESPORT,East McKeesport Borough,BOROUGH,Turtle Creek Valley,21496,821,31510,MV,0.378998,242.558563
127,PITTSBURGH,Pittsburgh,CITY,,61000,100,31908,PGH,NaN,NaN
128,PENN HILLS,Penn Hills Municipality,MUNICIPALI,Turtle Creek Valley,59032,934,35908,ES,19.462078,12455.728516


In [165]:
dataLoc = pd.merge(data, fromCounty[['from_ppt','from_LabelGIS']],  how='left', left_on=['13 - Where From'], right_on = ['from_ppt'])
print(dataLoc.columns)
dataLoc = pd.merge(dataLoc,
                    county_gis[['NAME', 'LABEL', 'TYPE', 'COG', 'FIPS', 'MUNICODE', 'OBJECTID']],
                    left_on=['from_LabelGIS'], right_on = ['LABEL'], suffixes = ('', '_from'))
print(dataLoc.columns)
dataLoc[['13 - Where From', 'from_LabelGIS', 'COG']]
dataLoc.drop(['from_ppt', 'NAME', 'LABEL', 'TYPE'], axis=1)
dataLoc = dataLoc.rename(columns={'COG': 'from_COG', 'FIPS': 'from_FIPS', 
                                  'MUNICODE': 'from_MUNICODE',
                                  'OBJECTID': 'from_OBJECTID'})
dataLoc['from_COG'].nunique()
dataLoc.to_csv('../Data/survey_fromclean.csv')

Index(['id', 'addr', 'timestamp', '3 - Area of interest',
       '4 - Communities of interest', '5 - Destinations',
       '6 - Corridor preference', '7 - Pittsburgh areas',
       '8 - Allegheny County areas', '9 - PAAC Routes Used',
       '10 - Routing Type Preference', '11 - Current Transit Usage',
       '12 - Other Transport Used', '12 - Why Other Transport Used',
       '13 - Age', '13 - Employment status', '13 - Where From', '14 - Gender',
       '14 - Race', '15 - Comments', 'from_ppt', 'from_LabelGIS'],
      dtype='object')
Index(['id', 'addr', 'timestamp', '3 - Area of interest',
       '4 - Communities of interest', '5 - Destinations',
       '6 - Corridor preference', '7 - Pittsburgh areas',
       '8 - Allegheny County areas', '9 - PAAC Routes Used',
       '10 - Routing Type Preference', '11 - Current Transit Usage',
       '12 - Other Transport Used', '12 - Why Other Transport Used',
       '13 - Age', '13 - Employment status', '13 - Where From', '14 - Gender',
       